In [ ]:
!pip install -q kaggle # install kaggle api client

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets list
!kaggle datasets download -d rupakroy/online-payments-fraud-detection-dataset

In [ ]:
!unzip online-payments-fraud-detection-dataset.zip

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

load dataset

In [ ]:
df = pd.read_csv('PS_20174392719_1491204439457_log.csv')

In [ ]:
df

eda

In [ ]:
df["nameOrig_type"]=df["nameOrig"].str[:1]

In [ ]:
df

In [ ]:
df=df.drop(columns=["nameOrig","nameOrig_type"])

In [ ]:
df["nameDest_type"]=df["nameDest"].str[:1]

In [ ]:
df

one hot encoding


In [ ]:
one_hot = pd.get_dummies(df[['nameDest_type', 'type']]) # Select multiple columns using a list within square brackets
df = df.drop(["nameDest_type", "type"], axis=1) # Pass columns to drop as a list
df = df.join(one_hot)

In [ ]:
df

In [ ]:
df = df.drop(["nameDest"], axis=1)

normalization

In [ ]:
def norm_column(df,col):
  df[col]=(df[col]-df[col].min())/(df[col].max()-df[col].min())


In [ ]:
norm_cols=[
    "step",
    "amount",
    "oldbalanceOrg",
    "newbalanceOrig",
    "oldbalanceDest",
    "newbalanceDest"
]

In [ ]:
for col in norm_cols:
  norm_column(df,col)

In [ ]:
df_isFraud=df[df["isFraud"]==1]
df_isnotFraud=df[df["isFraud"]==0]


In [ ]:
df_isFraud

undersampling(balancing)

In [ ]:
df_isnotfraud=df_isnotFraud.sample(n=8213, random_state=42)

In [ ]:
df_isnotfraud

In [ ]:
df=pd.concat([df_isFraud,df_isnotfraud])

In [ ]:
df

In [ ]:
# prompt: split data

from sklearn.model_selection import train_test_split

# Separate features and target
X = df.drop('isFraud', axis=1)
y = df['isFraud']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the train and test sets
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)


In [ ]:
import lightgbm as lgb
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
clf = lgb.LGBMClassifier()

clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
X_pred=clf.predict(X_train)
from sklearn.metrics import accuracy_score
accuracy_test = accuracy_score(y_test,y_pred)
print('\naccuracy_score on test dataset : ', accuracy_test)
precision_test = precision_score(y_test,y_pred)
print('\nprecision_score on test dataset : ', precision_test)
accuracy_test = accuracy_score(y_train,X_pred)
print('\naccuracy_score on train dataset : ', accuracy_test)
recall_test = recall_score(y_test,y_pred)
print('\nrecall_score on test dataset : ', recall_test)


confusion metrix

In [ ]:
from sklearn.metrics import confusion_matrix
conmat = confusion_matrix(y_test, y_pred)
import matplotlib.pyplot as plt
val = np.mat(conmat)

classnames = list(set(y_train))

df_cm = pd.DataFrame(

        val, index=classnames, columns=classnames,

    )

import seaborn as sns

plt.figure()

heatmap = sns.heatmap(df_cm, annot=True, cmap="Blues")

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right')

heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=0, ha='right')

In [ ]:
# plot

from matplotlib import pyplot as plt
# Plot feature importance using Gain
lgb.plot_importance(clf, importance_type="gain", figsize=(7,6), title="LightGBM Feature Importance (Gain)")
plt.show()
# Plot feature importance using Split
lgb.plot_importance(clf, importance_type="split", figsize=(7,6), title="LightGBM Feature Importance (Split)")
plt.show()


In [ ]:
!pip install shap
import shap
%time shap_values = shap.TreeExplainer(clf).shap_values(X_test)
shap.summary_plot(shap_values, X_test)
shap.dependence_plot("oldbalanceOrg", shap_values, X_test)
shap.dependence_plot("newbalanceOrig", shap_values,X_test )
shap.dependence_plot("nameDest_type_C", shap_values,X_test)
shap.dependence_plot("newbalanceDest", shap_values,X_test)
shap.dependence_plot("amount", shap_values,X_test)
shap.dependence_plot("type_TRANSFER", shap_values,X_test)
shap.dependence_plot("type_CASH_OUT", shap_values,X_test)

In [ ]:
import joblib

# Specify the file path where you want to save the model
model_path = 'model.pkl'

# Save the model to the specified file path
joblib.dump(clf, model_path)
